<a href="https://colab.research.google.com/github/fernando-marconi/Projeto-Pipeline-Combustivel/blob/main/Projeto_Pre%C3%A7os_Combustiveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto | Modelagem de dados**

In [ ]:
# Frameworks
import pandas as pd
import numpy as np
import duckdb
import os

from datetime import datetime

# Avisos
import warnings
warnings.filterwarnings('ignore')

# Driver
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Local do diretorio
diretorio = '/content/drive/MyDrive/Projeto_combustivel/Bronze/'
arquivos = os.listdir(diretorio)
arquivos

In [ ]:
# Criar o banco de dados
local_db = '/content/drive/MyDrive/Projeto_combustivel/Prata/'
conexao_banco = duckdb.connect( local_db + 'db_combustivel.db' )

In [ ]:
# Criando as tabelas no banco de dados

Query_Tabela_Dados = '''

CREATE TABLE IF NOT EXISTS tabela_combustiveis (
    regiao_sigla TEXT,
    estado_sigla TEXT,
    municipio TEXT,
    revenda TEXT,
    cnpj_da_revenda TEXT,
    nome_da_rua TEXT,
    numero_rua TEXT,
    complemento TEXT,
    bairro TEXT,
    cep TEXT,
    produto TEXT,
    data_da_coleta TIMESTAMP,
    valor_de_venda FLOAT,
    valor_de_compra FLOAT,
    unidade_de_medida TEXT,
    bandeira TEXT
);


'''

conexao_banco.execute( Query_Tabela_Dados )

In [ ]:
# Consulta para listar as tabelas existentes no banco de dados
tabelas_db = conexao_banco.execute('SHOW TABLES').fetchall()

# Exibindo as tabelas
print(tabelas_db)

In [ ]:
# Salvar os logs
tab_log = pd.DataFrame()

# Rodando o diretorio e lendo os arquivos
for Loop in arquivos[0:]:

  # verificando arquivo '.csv'
  if '.csv' in Loop:

    # Leitura dos dados
    #df = pd.read_csv( diretorio + Loop, sep=';', nrows=20, encoding='latin-1' )
    df = pd.read_csv( diretorio + Loop, sep=';', encoding='latin-1' )

    # Ajustando nome das colunas
    name_columns = [ loop.replace('-', '').replace('  ', '_').replace(' ', '_').replace('ï»¿', '').lower() for loop in df.columns ]
    df.columns = name_columns

    # Tratamento
    df.valor_de_venda = pd.to_numeric( df.valor_de_venda.replace(',', '.', regex=True), downcast='float' )
    df.valor_de_compra = pd.to_numeric( df.valor_de_compra.replace(',', '.', regex=True), downcast='float' )
    df.data_da_coleta = pd.to_datetime( df.data_da_coleta, format='%d/%m/%Y' )

    # Agora faça o append do DataFrame
    conexao_banco.append('tabela_combustiveis', df)

    # Criando um log de processamento
    tabela = df.dtypes.reset_index()
    campos_nulos = df.isnull().sum().values
    tabela['campos_nulos'] = campos_nulos
    tabela.columns = ['nome_campo', 'tipo_campo', 'campos_nulos']
    tabela['arquivo'] = Loop
    tabela['data'] = datetime.now().date()

    tab_log = pd.concat( [tab_log, tabela] )

    #print( df.columns )
    #print('')


In [ ]:
# Log de processamento
tab_log

In [ ]:
# Mandando pro Lake
tab_log.to_csv( '/content/drive/MyDrive/Projeto_combustivel/Prata/' + 'log_dados_combustivel.csv' )

In [ ]:
# Ler a tabela do banco de dados DuckDB

Query_Consulta = '''
SELECT *
FROM tabela_combustiveis

'''

# Conexão via DF
df_tabela = conexao_banco.execute( Query_Consulta ).fetchdf()
df_tabela.head()

In [ ]:
df_tabela.shape

In [ ]:
# Separar as datas
df_tabela['ano'] = df_tabela.data_da_coleta.dt.year
df_tabela['mes'] = df_tabela.data_da_coleta.dt.month
df_tabela['dia'] = df_tabela.data_da_coleta.dt.day
df_tabela.head()

In [ ]:
# Agrupamento
anl_tabela = df_tabela.groupby(
    by=['regiao_sigla', 'estado_sigla', 'municipio', 'produto', 'ano', 'mes'] ).agg(
      valor_medio_venda = ('valor_de_venda', 'mean')
).reset_index()

# Gerar uma Data
anl_tabela['data_mensal'] = pd.to_datetime( anl_tabela.ano.astype(int).astype(str) + '-' + anl_tabela.mes.astype(int).astype(str) + '-01' )

# Enriquecimento
dicionario = {
    'CO' : 'centro-oeste',
    'N' : 'norte',
    'NE' : 'nordeste',
    'SE' : 'sudeste',
    'S' : 'sul'
}
anl_tabela['regiao'] = anl_tabela.regiao_sigla.map(dicionario)

# Ajustar a ordem das colunas
anl_tabela = anl_tabela[['regiao', 'regiao_sigla', 'estado_sigla', 'municipio', 'data_mensal', 'ano', 'mes', 'produto', 'valor_medio_venda']]

# Convertendo nome do municipio
anl_tabela.municipio = anl_tabela.municipio.apply( lambda Loop : Loop.lower() )

# verificar
anl_tabela.head()

In [ ]:
anl_tabela.to_parquet( '/content/drive/MyDrive/Projeto_combustivel/Ouro/' + 'analitico_dados_combustivel.parquet' )

In [ ]:
# Importação da biblioteca de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Para gerar um gráfico claro, reagrupamos para ter a média nacional dos preços por mês/produto.
df_plot = anl_tabela.groupby(['data_mensal', 'produto']).agg(
    valor_medio_nacional = ('valor_medio_venda', 'mean')
).reset_index()

# Filtrar para os produtos mais comuns para uma visualização limpa
produtos_interesse = ['GASOLINA', 'DIESEL']
df_plot_final = df_plot[df_plot['produto'].isin(produtos_interesse)]

# --- Geração do Gráfico ---

# Configuração do tamanho da figura
plt.figure(figsize=(12, 6))

# Criação do gráfico de linhas, separando as cores por produto
sns.lineplot(
    data=df_plot_final,
    x='data_mensal',
    y='valor_medio_nacional',
    hue='produto',
    marker='o' # Adiciona um marcador em cada ponto de dados
)

# Adicionar rótulos e título
plt.title('Evolução do Preço Médio Nacional de Venda (2020-2025)', fontsize=16)
plt.xlabel('Data de Coleta (Mensal)', fontsize=12)
plt.ylabel('Preço Médio (R$ / Litro)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(title='Produto')
plt.xticks(rotation=45)
plt.tight_layout() # Ajusta o layout para evitar cortes

# Salvar o gráfico
plt.savefig('evolucao_valor_medio_venda.png')

In [ ]:
conexao_banco.close()